In [1]:
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# elem.send_keys(Keys.ENTER)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import re
import time
from datetime import datetime
import pickle

import pymysql 
import pandas as pd 
import numpy as np
from tqdm import tqdm

# # save data
# with open('urls.pickle','wb') as fw:
#     pickle.dump(myset.urls, fw)

# # load data
# with open('urls.pickle', 'rb') as fr:
#     urls = pickle.load(fr)

In [2]:
class createDB():
    def __init__(self, genres={'소설':'01', '시에세이':'03', '인문':'05', '경제경영':'13', 
                                '자기계발':'15', '정치사회':'17', '역사문화':'19', 
                                '예술대중문화':'23', '과학':'29' }):
        self.genres = genres
        self.urls, self.dfs = {}, {}
        
    def open_driver(self, url, scroll=False):
        user_agent = 'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
        options = webdriver.ChromeOptions()
        options.add_argument(user_agent)
        options.add_argument('headless') # 페이지 안 열기 
        options.add_argument('window-size=1920x1080') # 사이즈 지정
        options.add_argument("disable-gpu") # 안 보이게
        self.driver = webdriver.Chrome(options=options)
        self.driver.implicitly_wait(1)
        self.driver.get(url)
        time.sleep(0.5)
        
        if scroll:
            prev_height = self.driver.execute_script('return document.body.scrollHeight')
            while True:
                self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                time.sleep(1)
                curr_height = self.driver.execute_script('return document.body.scrollHeight')
                if prev_height == curr_height: break
                prev_height = curr_height
        
    def save_urls(self, genre):
        self.urls[genre] = {}
        tqdm_pages = tqdm(range(20,0,-1))
        for page in tqdm_pages: # 한 페이지 당 50권, 20페이지 반복(1000위까지)
            url = 'https://product.kyobobook.co.kr/category/KOR/{}#?page={}&type=best&per=50'.format(self.genres[genre], page)
            self.open_driver(url)
            best = self.driver.find_element_by_css_selector('#homeTabBest')
            elems = best.find_elements_by_class_name('prod_info_box')
            for i, elem in enumerate(elems):
                print('page-i:', page, i)
                try:
                    title = elem.find_element_by_css_selector('.prod_name').text
                    rank = elem.find_element_by_class_name('text').text
                    author = elem.find_element_by_css_selector('.prod_author > a').text
                    temp_url = elem.find_element_by_class_name('prod_info')
                    url = temp_url.get_attribute('href')
                    temp_rate = elem.find_element_by_class_name('review_klover_box')
                    rate = temp_rate.find_element(By.TAG_NAME, 'span').text
                    temp_count = elem.find_element_by_css_selector('.review_desc').text
                    count = re.sub(r'[^0-9]', '', temp_count)
                    
                    self.urls[genre][title] = {}
                    self.urls[genre][title]['url'] = url
                    self.urls[genre][title]['rank'] = int(rank)
                    self.urls[genre][title]['author'] = author
                    self.urls[genre][title]['rate'] = float(rate)
                    self.urls[genre][title]['count'] = int(count)
                except: print('pass')
                    
            tqdm_pages.set_description(genre+' '+str(page)+' urls')
        tqdm_pages.close()

    def connect_mysql(self, connect=True):
        if connect:
            self.db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', 
                                      charset='utf8', cursorclass=pymysql.cursors.DictCursor)
            self.cursor = self.db.cursor()
        else:
            self.db.commit()
            self.db.close
            
    def create_tbl(self, genre):
        self.connect_mysql()
        sql = """CREATE TABLE IF NOT EXISTS gyobo_{}(
                title VARCHAR(100), author VARCHAR(100),
                genre_1 VARCHAR(200), genre_2 VARCHAR(300), 
                brank SMALLINT, rate DOUBLE, count SMALLINT
                );""".format(genre)
        self.cursor.execute(sql)
        self.connect_mysql(False)
    
    def fill_tbl(self, genre):
        self.connect_mysql()
        tqdm_book = tqdm(self.urls[genre].items())
        for book_title, book_info in tqdm_book:
            try: self.open_driver(book_info['url'], scroll=True)
            except: continue
                
            title = book_title
#             title = self.driver.find_element_by_class_name('prod_title').text
            author, rank = book_info['author'], book_info['rank']
            rate = book_info['rate']
            count = book_info['count'] if book_info['count'] is not None else 'Null'
            
            elems = self.driver.find_elements_by_class_name('category_list_item')
            genre_1, genre_2 = '', ''
            for elem in elems:
                gs = elem.find_elements_by_class_name('intro_category_link')
                for i, g in enumerate(gs):
                    if i==1: genre_1 += g.text+', '
                    elif i==2 or i==3: genre_2 += g.text+', '
            
            sql = '''INSERT INTO gyobo_{0} VALUES(
            "{1}", "{2}", "{3}", "{4}", {5}, {6}, {7});
            '''.format(genre, title, author, genre_1, genre_2, rank, rate, count)
            self.cursor.execute(sql)
            self.db.commit()
            tqdm_book.set_description(genre+' insert data')
        self.connect_mysql(False)
        tqdm_book.close()
        
    def get_df(self, genre):
        self.connect_mysql(True)
        sql = 'SELECT * FROM gyobo_{}'.format(genre)
        self.cursor.execute(sql)
        self.dfs[genre] = pd.DataFrame(self.cursor.fetchall())
        self.dfs[genre].to_csv('gyobo_{}.csv'.format(genre), index=False)
        self.connect_mysql(False)
        print('save gyobo_'+genre+'.csv')
    
    def auto(self, genre, opt=True):
        if opt: self.save_urls(genre)
        self.create_tbl(genre)
        self.fill_tbl(genre)
        self.get_df(genre)

In [3]:
myset = createDB()
print(myset.genres)
# for genre in myset.genres.keys():
#     myset.auto(genre)

{'소설': '01', '시에세이': '03', '인문': '05', '경제경영': '13', '자기계발': '15', '정치사회': '17', '역사문화': '19', '예술대중문화': '23', '과학': '29'}


In [4]:
with open('urls_gyobo_novel_poem_inmun.pickle', 'rb') as fr:
    urls_1 = pickle.load(fr)
print(urls_1.keys())
    
with open('urls_gyobo_others.pickle', 'rb') as fr:
    urls_2 = pickle.load(fr)
print(urls_2.keys())

dict_keys(['소설', '시에세이', '인문'])
dict_keys(['경제경영', '자기계발', '정치사회', '역사문화', '예술대중문화', '과학'])


In [5]:
urls = urls_1
for g in urls_2.keys():
    urls[g] = urls_2[g]
for g in urls.keys():
    print(g, len(urls[g].keys()))

소설 863
시에세이 945
인문 890
경제경영 990
자기계발 943
정치사회 983
역사문화 998
예술대중문화 990
과학 978


In [6]:
with open('urls_gyobo.pickle','wb') as fw:
    pickle.dump(urls, fw)

In [ ]:
genre = ['소설','시에세이','인문'] # 863, 945, 890
with open('urls_gyobo.pickle', 'rb') as fr:
    urls = pickle.load(fr)
myset.urls = urls
for genre in urls.keys():
    myset.auto(genre, opt=False)

소설 insert data:  56%|████████████████████████████████▎                         | 481/863 [1:34:52<1:37:38, 15.34s/it]s]